In [ ]:
from ai_summarize import RevitAi


text: str = """
Um computador é uma máquina eletrônica que processa dados, realiza cálculos e executa tarefas complexas rapidamente. Ele é composto por hardware (componentes físicos) e software (programas) que, juntos, permitem manipulação de informações e automação de processos.
"""

tests = RevitAi()
results = tests.test_summarize(
    text
)

results

In [ ]:
from ai_summarize import RevitAi


data: dict = {
  "text": "League of Legends é um jogo online de estratégia e ação desenvolvido pela Riot Games, onde duas equipes de cinco jogadores competem para destruir a base adversária. Cada jogador controla um 'campeão' com habilidades únicas e trabalha em equipe para ganhar vantagens no mapa, derrotar oponentes e conquistar objetivos estratégicos. Popular no cenário de eSports, o jogo é conhecido por sua complexidade e alta competitividade."
}

text: str = data.get("text")

tests = RevitAi()
results = tests.summarize(
    text
)

results

In [4]:

import re

from transformers import AutoModelForSeq2SeqLM
from transformers import T5Tokenizer


MODEL_NAME: str = "csebuetnlp/mT5_multilingual_XLSum"
WHITESPACE_HANDLER = lambda k: re.sub(r'\s+', ' ', re.sub('\n+', ' ', k.strip()))

class RevitAi:
    
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)



    def chunk_text(self, text: str):
        # Se a quantidade de texto for muito grande, vamos tentar dividi-la por seções ou parágrafos
        paragraphs = text.split('\n')  # Dividindo por parágrafos

        chunk_size = 0
        chunks = []
        current_chunk = []

        for paragraph in paragraphs:
            current_chunk.append(paragraph)
            chunk_size += len(paragraph)

            # Se o tamanho do chunk ultrapassar um limite, armazene o chunk
            if chunk_size > 1500:  # Ajuste o limite conforme necessário
                chunks.append("\n".join(current_chunk))
                current_chunk = []
                chunk_size = 0

        # Adiciona o último pedaço se houver
        if current_chunk:
            chunks.append("\n".join(current_chunk))

        return chunks


    def summarize(self, text: str) -> str:
        
        input_ids = self.tokenizer(
            WHITESPACE_HANDLER(text),
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=2048
        )["input_ids"]
        
        output_ids = self.model.generate(
            input_ids=input_ids,
            max_length=2048,
            no_repeat_ngram_size=2,
            num_beams=4
        )[0]
        
        summary = self.tokenizer.decode(
            output_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )
        
        return summary



text = """
Os benefícios de uma cidade inteligente são muitos, incluindo redução de custos, melhora na
qualidade de vida, sustentabilidade ambiental e aumento da participação cívica. No entanto, os
desafios incluem garantir a segurança de dados, privacidade e reduzir desigualdades digitais.
"""

test = RevitAi()
test.summarize(text)

'A capital britânica, Londres, é uma cidade inteligente.'